In [8]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Instalar Framework de agentes do Google
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date, datetime
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, HTML # Para exibir a tabela formatada no Colab
import requests # Para fazer requisições HTTP
import warnings
import re
import pandas as pd

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

##########################################
# --- Agente 1: Analisador de Nascimento --- #
##########################################
def agente_analisador(data_nascimento):

    analisador = Agent(
        name="agente_analisador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista de personalidade e propósito de vida com base na data de nascimento.
        Sua tarefa é fornecer análises profundas e precisas sobre a personalidade, padrões emocionais,
        caminhos de carreira e desafios pessoais com base na data de nascimento fornecida.
        Use a ferramenta de busca do Google (google_search) para obter informações relevantes e
        garantir que as análises sejam fundamentadas e úteis.
        """,
        description="Agente que analisa a personalidade e o propósito de vida com base na data de nascimento",
        tools=[google_search]
    )

    entrada_do_agente_analisador = f"""
    Data de Nascimento: {data_nascimento}

    Realize as seguintes análises:

    1. Decodificador de Personalidade pela Data de Nascimento
    “Com base na data de nascimento {data_nascimento}, descreva meus pontos fortes naturais, padrões emocionais e como me comporto em relacionamentos — que seja profundo, específico e psicologicamente preciso.”

    2. Roteiro da Infância
    “Usando a data de nascimento {data_nascimento}, escreva um perfil psicológico de como minha infância moldou minha personalidade, hábitos e tomada de decisões hoje — seja gentil, mas revelador.”

    3. Analisador de Propósito Profissional
    “Dada a data de nascimento {data_nascimento}, quais caminhos de carreira combinam com meus traços de personalidade, valores e talentos naturais? Sugira áreas, funções e ambientes de trabalho.”

    4. Detector de Auto-Sabotagem
    “Com base na data {data_nascimento}, quais são meus hábitos de auto-sabotagem mais prováveis e como eles aparecem no dia a dia? Dê soluções práticas com base na psicologia.”

    5. Mapa de Gatilhos Emocionais
    “Usando a data de nascimento {data_nascimento}, explique o que geralmente me desencadeia emocionalmente, como eu costumo reagir e como posso desenvolver resiliência emocional em torno desses padrões.”

    6. Escaneamento de Energia nos Relacionamentos
    “Com base na data de nascimento {data_nascimento}, descreva como eu dou e recebo amor, o que preciso de um parceiro e que tipo de pessoa eu naturalmente atraio.”
    """

    analises = call_agent(analisador, entrada_do_agente_analisador)
    return analises

################################################
# --- Agente 2: Identificador de Melhorias --- #
################################################
def agente_melhorias(data_nascimento, analises_agente1):
    melhorias = Agent(
        name="agente_melhorias",
        model="gemini-2.0-flash",
        instruction="""
        Você é um consultor de desenvolvimento pessoal. Sua tarefa é analisar as análises fornecidas
        pelo Agente 1 (analisador de nascimento) e identificar áreas de melhoria em cada uma das seis
        categorias. Seja específico e forneça sugestões práticas para o desenvolvimento pessoal.
        """,
        description="Agente que identifica pontos de melhoria nas análises do Agente 1",
        tools=[google_search]
    )

    entrada_do_agente_melhorias = f"""
    Data de Nascimento: {data_nascimento}
    Análises do Agente 1: {analises_agente1}

    Para cada uma das seis análises fornecidas pelo Agente 1, identifique áreas de melhoria e
    forneça sugestões práticas para o desenvolvimento pessoal.
    """

    pontos_de_melhoria = call_agent(melhorias, entrada_do_agente_melhorias)
    return pontos_de_melhoria

######################################
# --- Agente 3: Buscador de Pessoas de Sucesso --- #
######################################
def agente_buscador_sucesso(data_nascimento):
    buscador_sucesso = Agent(
        name="agente_buscador_sucesso",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um pesquisador de pessoas de sucesso brasileiras. Sua tarefa é buscar na internet 5 homens e 5 mulheres
            que nasceram na mesma data fornecida e que alcançaram sucesso em suas áreas de atuação, e que sejam brasileiros.
            Monte uma tabela com as seguintes colunas: nome, profissão, no que a pessoa tem sucesso e site da informação.
            Ao realizar a busca no Google, certifique-se de incluir o termo "brasileiro" ou "brasileira" para garantir que os resultados sejam apenas de pessoas do Brasil.
            Use a ferramenta de busca do Google (google_search) para encontrar as informações e o site de onde tirou a informação.
            """,
        description="Agente que busca pessoas de sucesso nascidas na mesma data",
        tools=[google_search]
    )

    entrada_do_agente_buscador_sucesso = f"""
    Data de Nascimento: {data_nascimento}

    Busque na internet 5 homens e 5 mulheres que nasceram na mesma data e que alcançaram sucesso
    em suas áreas de atuação e que sejam brasileiros. Monte uma tabela com as seguintes colunas:
    nome, profissão, no que a pessoa tem sucesso e site da informação. Ao realizar a busca no Google, certifique-se de incluir o termo "brasileiro" ou "brasileira" para garantir que os resultados sejam apenas de pessoas do Brasil. Seja claro e objetivo
    """

    tabela_sucesso = call_agent(buscador_sucesso, entrada_do_agente_buscador_sucesso)

    # Extract data for DataFrame
    data_lines = tabela_sucesso.strip().split('\n')[1:]  # Skip header
    data = []
    for line in data_lines:
      values = re.split(r"\|", line)
      if len(values) >= 4:  # Ensure at least 4 parts in each line
        nome, profissao, sucesso, site = [v.strip() for v in values[:4]]
        data.append([nome, profissao, sucesso, site])

    # Create DataFrame
    df = pd.DataFrame(data, columns=["Nome", "Profissão", "Sucesso", "Site da Informação"])
    html_table = df.to_html(index=False)


    return html_table

##########################################
# --- Agente 4: Gerador de Relatório Final --- #
##########################################
def agente_relatorio_final(data_nascimento, analises, melhorias, tabela_sucesso):
    relatorio = Agent(
        name="agente_relatorio",
        model="gemini-2.0-flash",
        instruction="""
        Você é um gerador de relatórios finais. Sua tarefa é combinar as análises do Agente 1,
        os pontos de melhoria do Agente 2 e a tabela de pessoas de sucesso do Agente 3 em um
        relatório final otimista e motivador. Conclua o relatório com uma mensagem de incentivo.
        """,
        description="Agente que gera o relatório final",
        #tools=[google_search] # Remova a ferramenta de busca, pois não é necessária aqui
    )

    entrada_do_agente_relatorio = f"""
    Data de Nascimento: {data_nascimento}
    Análises do Agente 1: {analises}
    Pontos de Melhoria do Agente 2: {melhorias}
    Tabela de Pessoas de Sucesso do Agente 3: {tabela_sucesso}

    Combine as informações acima em um relatório final otimista e motivador.
    Conclua o relatório com uma mensagem de incentivo.
    """

    relatorio_final = call_agent(relatorio, entrada_do_agente_relatorio)
    return relatorio_final

##########################################
# --- Execução do Sistema de Agentes --- #
##########################################

print("🚀 Iniciando o Sistema de Análise de Personalidade e Propósito de Vida 🚀")

# --- Obter a Data de Nascimento do Usuário ---
while True:
    data_nascimento = input("❓ Por favor, digite sua DATA DE NASCIMENTO no formato DD/MM/AAAA: ")
    try:
        datetime.strptime(data_nascimento, '%d/%m/%Y')
        break  # Sai do loop se a data estiver no formato correto
    except ValueError:
        print("Formato de data incorreto. Use o formato DD/MM/AAAA.")

# Inserir lógica do sistema de agentes
if not data_nascimento:
    print("Você esqueceu de digitar a data de nascimento!")
else:
    print(f"Maravilha! Vamos analisar sua personalidade e propósito de vida com base em {data_nascimento}")

    analises_agente1 = agente_analisador(data_nascimento)
    print("\n--- 📝 Resultado do Agente 1 (Analisador de Nascimento) ---\n")
    display(to_markdown(analises_agente1))
    print("--------------------------------------------------------------")

    pontos_de_melhoria = agente_melhorias(data_nascimento, analises_agente1)
    print("\n--- 📝 Resultado do Agente 2 (Identificador de Melhorias) ---\n")
    display(to_markdown(pontos_de_melhoria))
    print("--------------------------------------------------------------")

    tabela_sucesso = agente_buscador_sucesso(data_nascimento)
    print("\n--- 📝 Resultado do Agente 3 (Buscador de Pessoas de Sucesso) ---\n")
    display(HTML(tabela_sucesso))
    print("--------------------------------------------------------------")

    relatorio_final = agente_relatorio_final(data_nascimento, analises_agente1, pontos_de_melhoria, tabela_sucesso)
    print("\n--- 📝 Resultado do Agente 4 (Gerador de Relatório Final) ---\n")
    display(to_markdown(relatorio_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Análise de Personalidade e Propósito de Vida 🚀
❓ Por favor, digite sua DATA DE NASCIMENTO no formato DD/MM/AAAA: 14/05/1968
Maravilha! Vamos analisar sua personalidade e propósito de vida com base em 14/05/1968

--- 📝 Resultado do Agente 1 (Analisador de Nascimento) ---



> Com base na sua data de nascimento (14/05/1968), aqui está uma análise da sua personalidade e propósito de vida:
> 
>  **1. Decodificador de Personalidade pela Data de Nascimento**
> 
>  *   **Signo do Zodíaco:** Touro. Taurinos são conhecidos por serem confiáveis, práticos e pacientes. Eles são criaturas de hábitos, gostam de estabilidade e apreciam as coisas boas da vida.
>  *   **Tratos de Personalidade:** Indivíduos nascidos em 14 de maio de 1968 possuem uma mentalidade filosófica, buscando conhecimento e compreensão. Eles são expressivos, apaixonados e não têm medo de compartilhar seus pensamentos e sentimentos. Gostam de discussões animadas e prosperam em ambientes onde podem se expressar livremente.
>  *   **Desafios Potenciais:** Essa mentalidade filosófica pode, às vezes, levar ao desapego e à falta de compromisso. A determinação pode se manifestar como teimosia, tornando-os inflexíveis e não dispostos a ceder quando suas decisões são desafiadas. Eles podem ficar inquietos e entediados se sentirem que estão presos em uma rotina.
>  *   **Numerologia:** Seu número do caminho de vida é 7. As pessoas com o número do caminho de vida 7 têm uma mente maravilhosa que ama penetrar na verdade de cada situação e, portanto, podem ser ótimos pesquisadores. Amam explorar todos os mistérios da vida e podem aplicar suas mentes a qualquer coisa que escolherem. Estão conectados ao Espírito em todas as coisas e têm uma intuição maravilhosa.
> 
>  **2. Roteiro da Infância**
> 
>  *   **Estabilidade Emocional e Desafios:** Quando crianças, os indivíduos nascidos em 14 de maio de 1968 eram conhecidos por serem emocionalmente estáveis. No entanto, eles poderiam recorrer a comportamentos autodestrutivos ao sentir problemas de apego.
>  *   **Saturno em Áries:** Seu Saturno está em Áries, o que significa que você luta contra agressão, impaciência, defensividade, pressa e presunção. Mas você tem um forte senso de autonomia e provavelmente atrairá desafios que o ajudarão a desenvolver autodisciplina.
> 
>  **3. Analisador de Propósito Profissional**
> 
>  *   **Traços Favoráveis:** As pessoas nascidas no dia 14 de qualquer mês são frequentemente descritas como charmosas, carismáticas e otimistas. Elas são naturalmente talentosas nas artes e têm um talento criativo que as diferencia dos outros.
>  *   **Caminhos de Carreira:** Dado o seu número de caminho de vida 7, você pode se destacar em áreas que exigem pesquisa, análise e resolução de problemas. Sua intuição e capacidade de ver entre as linhas podem ser valiosas em campos como ciência, filosofia ou investigação. Sua criatividade pode ser canalizada para as artes, design ou moda.
> *   **Adequação:** Indivíduos nascidos no ano do Macaco geralmente são atraídos por empreendimentos empresariais. A mentalidade inovadora e a adaptabilidade do Macaco os tornam adequados para trilhar seus caminhos, correr riscos e explorar empreendimentos criativos.
> 
>  **4. Detector de Auto-Sabotagem**
> 
>  *   **Possíveis Hábitos:** Dada a sua irritação com situações inalteradas, você pode sabotar-se ao evitar compromissos ou ficar inquieto em empregos ou relacionamentos estáveis. Você pode procrastinar ou fazer atalhos, o que pode levar a problemas com seus superiores.
>  *   **Soluções Práticas:** Reconheça a raiz da sua inquietação. É um desejo genuíno de crescimento ou um medo de compromisso?. Encontre maneiras de incorporar variedade e desafio em sua rotina. Defina metas realistas e divida-as em etapas menores para evitar se sentir sobrecarregado.
> 
>  **5. Mapa de Gatilhos Emocionais**
> 
>  *   **Gatilhos:** Ser colocado em situações inalteradas. Sentir-se preso ou confinado. Falta de autonomia e independência.
>  *   **Reações:** Inquietude, tédio e possível afastamento. Comportamentos autodestrutivos ao lidar com problemas de apego.
>  *   **Resiliência Emocional:** Pratique a autoconsciência para reconhecer quando esses gatilhos estão ativando suas emoções. Desenvolva mecanismos de enfrentamento saudáveis, como atenção plena, exercícios ou buscar apoio de outras pessoas. Concentre-se em aceitar o que você pode controlar e deixar de lado o que não pode.
> 
>  **6. Escaneamento de Energia nos Relacionamentos**
> 
>  *   **Como Você Dá e Recebe Amor:** Sendo um Taurino governado por Vênus, você provavelmente expressa amor de forma sensual e fundamentada. Você é leal e dedicado em relacionamentos, preferindo ir devagar.
>  *   **O Que Você Precisa de um Parceiro:** Estabilidade, lealdade e apreciação pelas coisas boas da vida. Um parceiro que lhe dê espaço para pensar e explorar e que respeite sua necessidade de autonomia.
>  *   **Que Tipo de Pessoa Você Atrai:** Por ser uma pessoa charmosa e carismática, você atrai indivíduos que são atraídos por sua criatividade e natureza compassiva. Você tende a se dar bem com Rato e Dragão. No entanto, podem surgir desafios em relacionamentos com Tigre e Cobra devido a diferentes temperamentos.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Identificador de Melhorias) ---



> Com base nas análises do Agente 1, aqui estão algumas áreas de melhoria e sugestões práticas para o seu desenvolvimento pessoal:
> 
> **1. Decodificador de Personalidade pela Data de Nascimento:**
> 
> *   **Área de Melhoria:** Lidar com a teimosia e a inflexibilidade.
> *   **Sugestões Práticas:**
>     *   **Pratique a escuta ativa:** Quando alguém desafiar sua opinião, tente realmente ouvir e entender o ponto de vista da outra pessoa antes de defender o seu.
>     *   **Questione suas próprias certezas:** Reserve um tempo para refletir sobre por que você acredita no que acredita. Esteja aberto à possibilidade de que você possa estar errado.
>     *   **Procure feedback:** Peça a amigos, familiares ou colegas para lhe darem feedback honesto sobre sua capacidade de ser flexível e de mente aberta.
> 
> **2. Roteiro da Infância:**
> 
> *   **Área de Melhoria:** Lidar com comportamentos autodestrutivos em resposta a problemas de apego.
> *   **Sugestões Práticas:**
>     *   **Terapia:** Considere procurar a ajuda de um terapeuta para explorar as raízes de seus problemas de apego e desenvolver mecanismos de enfrentamento mais saudáveis.
>     *   **Construa um sistema de apoio:** Conecte-se com amigos, familiares ou grupos de apoio que possam lhe oferecer apoio emocional e compreensão.
>     *   **Pratique o autocuidado:** Priorize atividades que lhe tragam alegria e ajudem você a se sentir calmo e relaxado. Isso pode incluir exercícios, meditação, passar tempo na natureza ou fazer algo criativo.
> 
> **3. Analisador de Propósito Profissional:**
> 
> *   **Área de Melhoria:** Encontrar um equilíbrio entre a necessidade de variedade e a necessidade de estabilidade.
> *   **Sugestões Práticas:**
>     *   **Procure um trabalho que ofereça oportunidades de crescimento e desenvolvimento:** Mesmo que você tenha um trabalho estável, procure maneiras de aprender coisas novas e desafiar-se.
>     *   **Incorpore variedade em sua rotina:** Encontre maneiras de quebrar a monotonia, mesmo que seja apenas experimentar um novo restaurante ou fazer um novo percurso para o trabalho.
>     *   **Defina metas de curto e longo prazo:** Isso ajudará você a se sentir mais no controle de sua vida e a progredir em direção a seus objetivos.
> 
> **4. Detector de Auto-Sabotagem:**
> 
> *   **Área de Melhoria:** Lidar com a inquietação e a tendência a evitar compromissos.
> *   **Sugestões Práticas:**
>     *   **Identifique seus gatilhos:** Preste atenção às situações que fazem você se sentir inquieto ou entediado.
>     *   **Desenvolva estratégias de enfrentamento:** Quando você se sentir tentado a sabotar-se, pare e reflita sobre as consequências de suas ações. Encontre maneiras mais saudáveis de lidar com seus sentimentos.
>     *   **Peça ajuda:** Se você está lutando para superar a auto-sabotagem, considere procurar a ajuda de um terapeuta ou coach.
> 
> **5. Mapa de Gatilhos Emocionais:**
> 
> *   **Área de Melhoria:** Desenvolver resiliência emocional para lidar com gatilhos emocionais.
> *   **Sugestões Práticas:**
>     *   **Mindfulness:** Pratique a atenção plena para aumentar a autoconsciência e observar seus pensamentos e sentimentos sem julgamento.
>     *   **Aceitação:** Aprenda a aceitar que você não pode controlar tudo. Concentre-se em aceitar o que você pode controlar e deixe de lado o que não pode.
>     *   **Autocompaixão:** Seja gentil consigo mesmo quando você estiver passando por momentos difíceis. Lembre-se de que todo mundo comete erros e que você não precisa ser perfeito.
> 
> **6. Escaneamento de Energia nos Relacionamentos:**
> 
> *   **Área de Melhoria:** Comunicar suas necessidades e expectativas em relacionamentos.
> *   **Sugestões Práticas:**
>     *   **Seja assertivo:** Expresse suas necessidades e desejos de forma clara e direta.
>     *   **Defina limites:** Não tenha medo de dizer não a coisas que você não quer fazer.
>     *   **Comunique-se abertamente:** Compartilhe seus pensamentos e sentimentos com seu parceiro.
> 
> Lembre-se de que o desenvolvimento pessoal é uma jornada contínua. Seja paciente consigo mesmo e celebre seus progressos ao longo do caminho.


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Buscador de Pessoas de Sucesso) ---



Nome,Profissão,Sucesso,Site da Informação
,Nome,Profissão,No que a pessoa tem sucesso
,-----------------,------------------------------,------------------------------------------------------------------
,Breno Silveira,"Diretor de fotografia, produtor, diretor",Cinema
,Sérgio Guizé,Ator,Atuação em novelas e séries
,Raul Pont,Político,Atuação na política brasileira
,João Acaiabe,Ator,Atuação em televisão
,Luciano Szafir,Ator,Atuação em televisão
,Petra Leão,Quadrinista brasileira,História em quadrinhos
,Vanessa Barum,Cantora brasileira,Cantora
,Jandyra Planet do Amaral,Médica,"Uma das primeiras médicas brasileiras, pioneira na área científica no Instituto Butantan"


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Gerador de Relatório Final) ---



> ## Relatório Final: Uma Jornada de Descoberta e Realização
> 
> Prezado(a) visionário(a),
> 
> Este relatório final é o culminar de análises profundas, insights valiosos e um olhar atento ao seu potencial ilimitado. Com base na sua data de nascimento e nas análises detalhadas dos Agentes 1 e 2, estamos entusiasmados em apresentar um roteiro personalizado para o seu sucesso e bem-estar.
> 
> **Sua Essência e Propósito:**
> 
> Como um taurino com o número de caminho de vida 7, você é um ser de estabilidade, praticidade e profunda intuição. Sua mente filosófica e talento criativo o(a) impulsionam a buscar conhecimento e a expressar suas ideias de forma apaixonada. No entanto, reconhecemos os desafios que podem surgir, como a teimosia e a tendência à inquietação.
> 
> **Oportunidades de Crescimento:**
> 
> A boa notícia é que você tem o poder de transformar esses desafios em trampolins para o sucesso. Ao praticar a escuta ativa, questionar suas certezas e buscar feedback honesto, você pode cultivar a flexibilidade e a abertura mental. Além disso, ao abordar os problemas de apego com terapia e construir um sistema de apoio sólido, você pode superar comportamentos autodestrutivos e fortalecer sua resiliência emocional.
> 
> **Carreira e Realização:**
> 
> Seu talento para pesquisa, análise e resolução de problemas, combinado com sua intuição aguçada, o(a) torna um(a) candidato(a) ideal para áreas como ciência, filosofia, investigação, artes, design ou moda. A adaptabilidade e a mentalidade inovadora associadas ao ano do Macaco também o(a) tornam um(a) empreendedor(a) natural.
> 
> **Relações Harmoniosas:**
> 
> Nos relacionamentos, sua lealdade, dedicação e apreciação pelas coisas boas da vida são qualidades valiosas. Ao comunicar suas necessidades e expectativas de forma clara e assertiva, você pode construir conexões mais fortes e satisfatórias.
> 
> **Inspiração em Pessoas de Sucesso:**
> 
> Assim como Breno Silveira, Sérgio Guizé, Raul Pont e tantos outros nascidos em 14 de maio, você tem o potencial de alcançar grandes feitos em sua área de atuação. Inspire-se em suas histórias de sucesso e siga em frente com confiança e determinação.
> 
> **Mensagem de Incentivo:**
> 
> Lembre-se de que o desenvolvimento pessoal é uma jornada contínua, repleta de altos e baixos. Seja paciente consigo mesmo(a), celebre seus progressos e nunca perca de vista seus sonhos. Acredite em seu potencial ilimitado e siga em frente com coragem e paixão. O mundo precisa do seu talento e da sua visão única!
> 
> Com entusiasmo e otimismo,
> 
> Sua equipe de apoio ao sucesso.
> 


--------------------------------------------------------------
